In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('spam.csv',encoding='ISO-8859-1')

In [3]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


# Data Cleaning

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 108.9+ KB


In [5]:
#drop last three columns
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)

In [6]:
df.sample(5)

,v1,v2
1702,ham,I have lost 10 kilos as of today!
1260,ham,"Yo, I'm at my parents' gettin cash. Good news:..."
344,ham,My sister cleared two round in birla soft yest...
4942,ham,Check mail.i have mailed varma and kept copy t...
2943,spam,We currently have a message awaiting your coll...


In [7]:
df.rename(columns={'v1':'target','v2':'text'},inplace=True)
df.sample(5)

,target,text
388,spam,4mths half price Orange line rental & latest c...
1498,ham,Time n Smile r the two crucial things in our l...
3081,ham,"And very importantly, all we discuss is betwee..."
572,ham,Can you open the door?
402,ham,The hair cream has not been shipped.


In [8]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [9]:
df['target'] = encoder.fit_transform(df['target'])

In [10]:
df.head()

,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [11]:
df.isnull().sum()

target    0
text      0
dtype: int64

In [12]:
df.duplicated().sum()

403

In [13]:
df = df.drop_duplicates(keep='first')

In [14]:
df.duplicated().sum()

0

In [15]:
df.shape

(5169, 2)

In [16]:
df['target'].value_counts()

0    4516
1     653
Name: target, dtype: int64

In [17]:
df_copy = df['text'].copy()

In [18]:
import string
from nltk.corpus import stopwords
def text_preprocess(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    return " ".join(text)


In [19]:
df_copy = df_copy.apply(text_preprocess)

In [20]:
df_copy

0       Go jurong point crazy Available bugis n great ...
1                                 Ok lar Joking wif u oni
2       Free entry 2 wkly comp win FA Cup final tkts 2...
3                     U dun say early hor U c already say
4             Nah dont think goes usf lives around though
                              ...                        
5567    2nd time tried 2 contact u U å£750 Pound prize...
5568                          Ì b going esplanade fr home
5569                          Pity mood Soany suggestions
5570    guy bitching acted like id interested buying s...
5571                                       Rofl true name
Name: text, Length: 5169, dtype: object

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [22]:
vectorizer = TfidfVectorizer()

In [23]:
df_mat = vectorizer.fit_transform(df_copy)
df_mat

<5169x9376 sparse matrix of type '<class 'numpy.float64'>'
	with 43208 stored elements in Compressed Sparse Row format>

In [24]:
message_train, message_test, spam_nospam_train, spam_nospam_test = train_test_split(df_mat, 
                                                        df['target'], test_size=0.3, random_state=20)

# Model

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [26]:
lb_model = LogisticRegression(solver='liblinear', penalty='l1')
lb_model.fit(message_train, spam_nospam_train)
pred = lb_model.predict(message_test)
accuracy_score(spam_nospam_test,pred)

0.941972920696325

In [27]:
def stemmer (text):
    text = text.split()
    words = ""
    for i in text:
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "
    return words

In [28]:
from nltk.stem import SnowballStemmer
df_copy = df_copy.apply(stemmer)
vectorizer = TfidfVectorizer()
df_mat = vectorizer.fit_transform(df_copy)

In [29]:
message_train, message_test, spam_nospam_train, spam_nospam_test = train_test_split(df_mat, 
                                                        df['target'], test_size=0.3, random_state=20)

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lb_model = LogisticRegression(solver='liblinear', penalty='l1')
lb_model.fit(message_train, spam_nospam_train)
pred = lb_model.predict(message_test)
accuracy_score(spam_nospam_test,pred)

0.9477756286266924

In [31]:
df['length'] = df['text'].apply(len)
df.head()

,target,text,length
0,0,"Go until jurong point, crazy.. Available only ...",111
1,0,Ok lar... Joking wif u oni...,29
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,0,U dun say so early hor... U c already then say...,49
4,0,"Nah I don't think he goes to usf, he lives aro...",61


In [32]:
length = df['length'].to_numpy()
new_mat = np.hstack((df_mat.todense(),length[:, None]))

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lb_model = LogisticRegression(solver='liblinear', penalty='l1')
lb_model.fit(message_train, spam_nospam_train)
pred = lb_model.predict(message_test)
accuracy_score(spam_nospam_test,pred)

0.9477756286266924

In [34]:
import pickle
pickle.dump(vectorizer,open('vectorizer_1.pkl','wb'))
pickle.dump(lb_model,open('model_1.pkl','wb'))